# COVID mortality rate by testing

In [2]:
library(rvest)
library(data.table)
library(stringr)
library(meta)

In [3]:
# timestamp for saving the file. 
timestamp  = Sys.time()

cases_url = 'https://www.worldometers.info/coronavirus/'
cases = read_html(cases_url)  
casedata = html_nodes(cases, '#main_table_countries_today')
cdt = html_table(casedata)

# extract the updating time 
timest = html_nodes(cases,'div:nth-child(5)')
ts = html_text(timest)[[1]]

In [4]:
# turn the data into a data.table object
cdt = setDT(cdt[[1]])
setnames(cdt,old = 'Country,Other', new = 'Country')
cdt[,TotalDeaths:=as.numeric(str_replace(TotalDeaths,",",""))]
cdt[,TotalCases:=as.numeric(str_replace(TotalCases,",",""))]

In [5]:
# change character to numeric variables 
cdt[,TotalDeaths:=as.numeric(TotalDeaths)] 
cdt[,NewDeaths:=as.numeric(NewDeaths)]
cdt[is.na(TotalDeaths), TotalDeaths:=0]
cdt[is.na(NewDeaths), NewDeaths:=0]
cdt[,CFR:=((TotalDeaths)/TotalCases)*100, by = Country]
cdt[Country=="S. Korea", Country:="South Korea"]

In [6]:
# Exclusions 
cdt2 = cdt[TotalDeaths > 3]
cdt3 = cdt2[!{Country=="Diamond Princess" | Country=="Total:"}]
cdt3

Country,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,CFR
<chr>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>
China,81008,+41,3255,7,"71,740","6,013","1,927",56,4.0181217
Italy,47021,,4032,0,"5,129","37,860","2,655",778,8.5748921
Spain,24926,"+3,355",1326,233,"2,125","21,475",939,533,5.3197464
Iran,20610,+966,1556,123,"7,635","11,419",,245,7.5497331
Germany,20581,+733,72,4,209,"20,300",2,246,0.3498372
USA,19775,+392,276,20,147,"19,352",64,60,1.3957016
France,12612,,450,0,"1,587","10,575","1,297",193,3.5680304
South Korea,8799,+147,102,8,"2,612","6,085",59,172,1.1592226
Switzerland,6186,+571,58,2,15,"6,113",141,715,0.9376010


In [7]:
# Change the layour of the confidence interval 
cilayout(bracket = "(", separator = " to ")

In [8]:
# Run the meta-analysis 
MA = metaprop(event = TotalDeaths, 
              n = TotalCases, 
              data = cdt3,
              studlab = Country,
              comb.random = F, 
              prediction  = T,
              method = "Inverse",
)

In [9]:
fnm = paste0("CFR_analysis_",substring(timestamp,0,10),".png")
png(filename = fnm, res = 300, height= 2400, width = 3000)
forest(MA,sortvar = -cdt3$CFR,prediction = T,leftlabs = c("Country","Deaths","Cases"), digits.tau2 = 2, 
       pscale = 100, rightlabs = c("Case Fatality (%)","95%-CI"),#xlim=c(0,45), 
       overall = F, colgap.left = "4mm",fontsize = 10,
       text.addline1 = "\n", text.addline2 = paste0(ts), 
       ff.addline = "bold")

dev.off()
# Copy to current file name
file.copy(fnm, "CFR_analysis.png")

png 
  2

[1] TRUE

![title](./CFR_analysis.png)